#Chapter 2 exercise

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

##Downloading and Loading data

In [2]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [3]:
fetch_housing_data()

In [4]:
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [5]:
housing_data = load_housing_data()

In [6]:
housing_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### Adding some extra attributes 

In [7]:
from sklearn.base import TransformerMixin , BaseEstimator
attrs = ['total_rooms' ,'total_bedrooms', 'population' ,'households' ]
room_ix , bedrooms_ix , households_ix, population_ix = (housing_data.columns.get_loc(i) for i in attrs)  
print(room_ix , bedrooms_ix , households_ix, population_ix)
class CombinerAttrAdder(BaseEstimator , TransformerMixin):
  def __init__(self, add_bedrooms_per_room =False):
    self.add_bedrooms_per_room = add_bedrooms_per_room
  def fit(self, X, y= None):
    return self
  def transform(self , X):
    pop_per_household = X[:,population_ix]/X[:,households_ix]
    rooms_per_household = X[:,room_ix]/X[:,households_ix]
    if self.add_bedrooms_per_room == True:
      bedrooms_per_rooms = X[:,bedrooms_ix]/X[:,room_ix]
      return np.c_[X ,pop_per_household,rooms_per_household , bedrooms_per_rooms]
    else:
      return np.c_[X ,pop_per_household,rooms_per_household ]


3 4 5 6


## Spliting the data and making pipelines

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
housing = housing_data.drop('median_house_value', axis = 1)
housing_label = housing_data['median_house_value']

X_train ,X_test ,y_train , y_test = train_test_split(housing, housing_label , test_size = 0.2 , random_state = 7)

In [11]:
num_pipeline = Pipeline([('imputer' , SimpleImputer(strategy = 'median')),
                         ('atts_adder', CombinerAttrAdder(add_bedrooms_per_room= True)),
                         ('scaler', StandardScaler())])

In [12]:
housing_num =  housing.drop('ocean_proximity' , axis =1 ).columns

In [13]:
full_pipeline = ColumnTransformer([('num' , num_pipeline , list(housing_num)),
                                   ('cat' , OneHotEncoder() , ['ocean_proximity'])])

#Exercise

###Q-1

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

###Grid Search on SVR 

In [ ]:
param_grid = [{'kernel':['linear'] , 'C':[5,50,500,5000] },
              {'kernel':['rbf'] , 'C': [5,50,500,5000] , 'gamma':['auto', 'scale'] }]

housing_prepared = full_pipeline.fit_transform(X_train)

grd_svr = GridSearchCV(SVR() , param_grid= param_grid , scoring = 'neg_mean_squared_error' , 
                       cv =5 , n_jobs =-1 ,return_train_score= True , verbose =1)

grd_svr.fit(housing_prepared, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 14.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'C': [5, 50, 500, 5000], 'kernel': ['linear']},
                         {'C': [5, 50, 500, 5000], 'gamma': ['auto', 'scale'],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=1)

In [ ]:
for params , score in zip(grd_svr.cv_results_['params'] , grd_svr.cv_results_['mean_test_score']):
  print(params , np.sqrt(-score))

{'C': 5, 'kernel': 'linear'} 90992.07478735418
{'C': 50, 'kernel': 'linear'} 69167.8543617127
{'C': 500, 'kernel': 'linear'} 67360.65629753521
{'C': 5000, 'kernel': 'linear'} 67203.51053744886
{'C': 5, 'gamma': 'auto', 'kernel': 'rbf'} 116740.12287695573
{'C': 5, 'gamma': 'scale', 'kernel': 'rbf'} 116743.64152843636
{'C': 50, 'gamma': 'auto', 'kernel': 'rbf'} 104244.1001614259
{'C': 50, 'gamma': 'scale', 'kernel': 'rbf'} 104360.29006871692
{'C': 500, 'gamma': 'auto', 'kernel': 'rbf'} 72456.63105297046
{'C': 500, 'gamma': 'scale', 'kernel': 'rbf'} 73605.60198454004
{'C': 5000, 'gamma': 'auto', 'kernel': 'rbf'} 60061.385233217545
{'C': 5000, 'gamma': 'scale', 'kernel': 'rbf'} 59722.89112383708


In [ ]:
grd_svr.best_params_ , np.sqrt(-grd_svr.best_score_) 

({'C': 5000, 'gamma': 'scale', 'kernel': 'rbf'}, 59722.89112383708)

In [ ]:
svr = SVR(**grd_svr.best_params_)
svr.fit(housing_prepared , y_train )

SVR(C=5000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
y_pred = svr.predict(housing_prepared)
rmse = mean_squared_error(y_train,y_pred , squared = False)
print(rmse)

58687.42806078929


##Observation 

SVM seems to perform inferior to RandomForest, the best C is 5000, so requires very less regularization and the kernel is linear.

##Q-2

### Random Search CV on SVR

In [ ]:
%%time
from scipy.stats import expon, reciprocal
from scipy.stats import randint

#The use of different distributions for hyperparameter tuning is taken for the official answers

param_dist = {
        'kernel': ['linear', 'rbf'],
        'C': randint(2000,15000),
        'gamma': expon(scale = 1),
    }

svm_reg = SVR()
rnd_svr = RandomizedSearchCV(svm_reg, param_distributions=param_dist,
                                n_iter=15, cv=5, scoring='neg_mean_squared_error',
                                verbose=1 , n_jobs = -1)
rnd_svr.fit(housing_prepared, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 12.7min finished


CPU times: user 27.3 s, sys: 521 ms, total: 27.8 s
Wall time: 13min 7s


In [ ]:
for params , score in zip(rnd_svr.cv_results_['params'] , rnd_svr.cv_results_['mean_test_score']):
  print(params , np.sqrt(-score))


{'C': 14279, 'gamma': 0.3554451278997717, 'kernel': 'rbf'} 58776.58077335331
{'C': 6790, 'gamma': 0.5238787555432229, 'kernel': 'rbf'} 68955.35506023302
{'C': 4844, 'gamma': 0.8816510979845603, 'kernel': 'linear'} 67205.0045842991
{'C': 5986, 'gamma': 1.2322621605734507, 'kernel': 'linear'} 67194.87725786293
{'C': 13288, 'gamma': 2.016005251560359, 'kernel': 'linear'} 67172.87801316072
{'C': 4187, 'gamma': 4.269507409610038, 'kernel': 'linear'} 67214.34469883611
{'C': 9750, 'gamma': 0.9799352165592159, 'kernel': 'linear'} 67179.56957002883
{'C': 6805, 'gamma': 0.18291011293107068, 'kernel': 'linear'} 67188.8547542652
{'C': 10231, 'gamma': 0.34624295233188856, 'kernel': 'linear'} 67175.82158648936
{'C': 7688, 'gamma': 4.405501825075636, 'kernel': 'linear'} 67184.76403147739
{'C': 14614, 'gamma': 0.911464465816738, 'kernel': 'rbf'} 72081.15668949255
{'C': 4666, 'gamma': 0.5311638095995389, 'kernel': 'rbf'} 73085.04205075216
{'C': 14680, 'gamma': 1.626758452264275, 'kernel': 'rbf'} 85527.

In [ ]:
rnd_svr.best_params_ , np.sqrt(-rnd_svr.best_score_)

({'C': 14279, 'gamma': 0.3554451278997717, 'kernel': 'rbf'}, 58776.58077335331)

In [ ]:
svr = SVR(**rnd_svr.best_params_)
svr.fit(housing_prepared , y_train )

SVR(C=14279, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=0.3554451278997717, kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred = svr.predict(housing_prepared)
rmse = mean_squared_error(y_train,y_pred , squared = False)
print(rmse)

54726.53111460167


In [ ]:
import joblib
import pickle
# saving both the models with the CV results 
joblib.dump(rnd_svr, "rnd_svr1.pkl") 

joblib.dump(grd_svr, "grd_svr1.pkl") 

#my_model_loaded = joblib.load("grd_svr.pkl")

['grd_svr1.pkl']

In [15]:
import joblib
rnd_svr , grd_svr = joblib.load('/content/drive/MyDrive/rnd_svr1.pkl') , joblib.load('/content/drive/MyDrive/grd_svr1.pkl')

##Observations

C is now increased to almost 15000, so the regularization strength is reduced further. kernel is still rbf.

#Q-3

To add another transformer for selecting the important attributes, I am training Random Forest because it performed better than SVR, and will use the feature importance of the Random Forests model.

In [ ]:
param_grid = [{'n_estimators': [5,10,50, 100 , 500], 'max_features': [2, 4, 6, 8]}]

housing_prepared = full_pipeline.fit_transform(X_train)
grd_rf = GridSearchCV(RandomForestRegressor() , param_grid , scoring = 'neg_mean_squared_error', 
                      cv = 7 , n_jobs = -1, return_train_score= 'True' , verbose = 1)
grd_rf.fit(housing_prepared, y_train)

Fitting 7 folds for each of 20 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed: 13.6min finished


GridSearchCV(cv=7, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [ ]:
for params , score in zip(grd_rf.cv_results_['params'] , grd_rf.cv_results_['mean_test_score']):
  print(params , np.sqrt(-score))

{'max_features': 2, 'n_estimators': 5} 57863.947204840886
{'max_features': 2, 'n_estimators': 10} 54502.47834409041
{'max_features': 2, 'n_estimators': 50} 51252.27855347033
{'max_features': 2, 'n_estimators': 100} 50624.35806545301
{'max_features': 2, 'n_estimators': 500} 50448.26473995753
{'max_features': 4, 'n_estimators': 5} 55541.60081025265
{'max_features': 4, 'n_estimators': 10} 51358.61499336446
{'max_features': 4, 'n_estimators': 50} 48941.295960971256
{'max_features': 4, 'n_estimators': 100} 48766.75152340891
{'max_features': 4, 'n_estimators': 500} 48437.72097298398
{'max_features': 6, 'n_estimators': 5} 54408.16989759661
{'max_features': 6, 'n_estimators': 10} 51297.48921106692
{'max_features': 6, 'n_estimators': 50} 48772.78197075231
{'max_features': 6, 'n_estimators': 100} 48378.301081310856
{'max_features': 6, 'n_estimators': 500} 48103.961855417096
{'max_features': 8, 'n_estimators': 5} 54417.46456704444
{'max_features': 8, 'n_estimators': 10} 50999.908015216264
{'max_f

In [ ]:
feat_imp = grd_rf.best_estimator_.feature_importances_
feat_imp

array([7.40345171e-02, 7.24855917e-02, 3.90956895e-02, 1.68808238e-02,
       1.58729942e-02, 1.60815552e-02, 1.61885193e-02, 3.23418064e-01,
       7.16049097e-02, 1.12473272e-01, 7.27868920e-02, 1.25728870e-02,
       1.48652947e-01, 1.23582304e-04, 3.21114984e-03, 4.51660592e-03])

In [ ]:
extra_attrs =['pop_per_household','rooms_per_household' , 'bedrooms_per_rooms' ]

In [ ]:
cat_ohe = full_pipeline.named_transformers_['cat']

cat_attr = list(cat_ohe.categories_[0])

num_attrs = list(housing.drop('ocean_proximity' , axis = 1).columns)
att = num_attrs + extra_attrs + cat_attr

sorted(zip(feat_imp , att) , reverse = True)

[(0.32341806415034574, 'median_income'),
 (0.14865294682201835, 'INLAND'),
 (0.1124732715025282, 'rooms_per_household'),
 (0.07403451710713581, 'longitude'),
 (0.07278689195807725, 'bedrooms_per_rooms'),
 (0.07248559174281992, 'latitude'),
 (0.07160490967785187, 'pop_per_household'),
 (0.03909568947053962, 'housing_median_age'),
 (0.016880823777769643, 'total_rooms'),
 (0.016188519310159216, 'households'),
 (0.016081555162714745, 'population'),
 (0.015872994207297268, 'total_bedrooms'),
 (0.012572887046521415, '<1H OCEAN'),
 (0.004516605924345549, 'NEAR OCEAN'),
 (0.0032111498361270014, 'NEAR BAY'),
 (0.00012358230374848636, 'ISLAND')]

In [ ]:
rf = RandomForestRegressor(**grd_rf.best_params_)

In [ ]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(rf, housing_prepared, y_train,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)


In [ ]:
forest_rmse_scores

array([45561.72069812, 49703.31485123, 47300.42215056, 46656.89702172,
       47275.71994999, 50989.50865197, 47220.37546159, 49784.11609367,
       47055.3572604 , 48166.36363249])

In [ ]:
import joblib
joblib.dump(grd_rf, 'grd_rf.pkl')

['grd_rf.pkl']

In [17]:
grd_rf = joblib.load('/content/drive/MyDrive/grd_rf.pkl')

In [18]:
feat_imp = grd_rf.best_estimator_.feature_importances_

In [19]:
class TopKfeatures(BaseEstimator , TransformerMixin):
  def __init__(self, feature_imp , k):
    self.k = k
    self.feature_imp = feature_imp
  def fit(self ,X,y=None):

    self.top_k = np.argsort(self.feature_imp)[::-1][:self.k]
    return self
  def transform(self, X):
    return X[:,self.top_k]

In [20]:
top_feat_pipeline = Pipeline([('full' , full_pipeline),
                              ('top_k_feat' , TopKfeatures(feat_imp, 10))])

In [21]:
housing_prep_new = top_feat_pipeline.fit_transform(X_train)

In [22]:
svr_top_k = SVR(**rnd_svr.best_params_)
#r = svr_top_k.best_estimator_ #housing_prep_new
svr_top_k.fit(housing_prep_new , y_train)

SVR(C=14279, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=0.3554451278997717, kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [23]:
svr_top_pred = svr_top_k.predict(housing_prep_new )


svr_rmse = mean_squared_error(y_train, svr_top_pred , squared = False)
svr_rmse

54339.63874754279

##Observations 
Using only 10 features we still get a RMSE value 54339 comparable to RMSE using all features, The median income is the most important feature.

#Q-4 

In [24]:
pipeline_with_predict = Pipeline([('preparation' , full_pipeline),
                              ('top_k_feat' , TopKfeatures(feat_imp,k =10)),
                              ('prediction' , SVR(**rnd_svr.best_params_))])

In [25]:
pipeline_with_predict.fit(X_train , y_train)

Pipeline(memory=None,
         steps=[('preparation',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                              

In [26]:
y_pred_train = pipeline_with_predict.predict(X_train)

In [27]:
y_pred_test = pipeline_with_predict.predict(X_test)

In [28]:
train_mse = mean_squared_error(y_train , y_pred_train , squared =False)
test_mse = mean_squared_error(y_test , y_pred_test , squared =False)
print(train_mse , test_mse)

54339.63874754279 58990.44678064234


#Q-5

In [ ]:
param_grid = [{
    'preparation__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
    'top_k_feat__k': list(range(1, len(feat_imp) + 1))
}]


grd_from_pipeline =GridSearchCV(pipeline_with_predict , param_grid , scoring ='neg_mean_squared_error',
                                cv = 5 , n_jobs = -1 ,verbose = 1 , return_train_score = 'True')
grd_from_pipeline.fit(X_train , y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 46.8min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 58.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preparation',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('imputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                                     

In [ ]:
for params , score in zip(grd_from_pipeline.cv_results_['params'] , grd_from_pipeline.cv_results_['mean_test_score']):
  print(params , np.sqrt(-score))

{'preparation__num__imputer__strategy': 'mean', 'top_k_feat__k': 1} 84066.00320647444
{'preparation__num__imputer__strategy': 'mean', 'top_k_feat__k': 2} 75293.02033080747
{'preparation__num__imputer__strategy': 'mean', 'top_k_feat__k': 3} 69081.19088180165
{'preparation__num__imputer__strategy': 'mean', 'top_k_feat__k': 4} 65167.08167762155
{'preparation__num__imputer__strategy': 'mean', 'top_k_feat__k': 5} 61665.12718756939
{'preparation__num__imputer__strategy': 'mean', 'top_k_feat__k': 6} 58405.890563248475
{'preparation__num__imputer__strategy': 'mean', 'top_k_feat__k': 7} 58447.82166651119
{'preparation__num__imputer__strategy': 'mean', 'top_k_feat__k': 8} 55702.6320427275
{'preparation__num__imputer__strategy': 'mean', 'top_k_feat__k': 9} 56678.41780190337
{'preparation__num__imputer__strategy': 'mean', 'top_k_feat__k': 10} 57458.60703402374
{'preparation__num__imputer__strategy': 'mean', 'top_k_feat__k': 11} 58006.36918188319
{'preparation__num__imputer__strategy': 'mean', 'top

In [ ]:
grd_from_pipeline.best_params_ , np.sqrt(-grd_from_pipeline.best_score_)

({'preparation__num__imputer__strategy': 'mean', 'top_k_feat__k': 8},
 55702.6320427275)

In [ ]:
joblib.dump(grd_from_pipeline, 'grd_pipeline_svr1.pkl')

['grd_pipeline_svr1.pkl']

In [29]:
grd_pipeline_svr1 = joblib.load('/content/drive/MyDrive/grd_pipeline_svr1.pkl')

In [35]:
full_pipeline__num__imputer_strategy = 'mean'

In [36]:
pipeline_with_predict = Pipeline([('preparation' , full_pipeline),
                              ('top_k_feat' , TopKfeatures(feat_imp,k =8)),
                              ('prediction' , SVR(**rnd_svr.best_params_))])

In [37]:
pipeline_with_predict.fit(X_train, y_train)
y_pred_train = pipeline_with_predict.predict(X_train) 
y_pred_test = pipeline_with_predict.predict(X_test)

In [38]:
train_mse = mean_squared_error(y_train , y_pred_train , squared =False)
test_mse = mean_squared_error(y_test , y_pred_test , squared =False)
print(train_mse , test_mse)

53812.90364797498 57354.41188368425


##Observations 

Using 8 features and strategy as mean seems to work best. 